In [2]:
# Imports and data reading from files
import pandas as pd

data = pd.read_csv('../../data/games/2021-22.csv')
players = pd.read_csv('../../data/players/2021-22.csv')
teams = pd.read_csv('../../data/teams.csv')

# Splitting raw teams data into east and west conferences
conferences = { 
    'east' : list(teams[teams['location'] == 'east']['team']),
    'west' : list(teams[teams['location'] == 'west' ]['team'])
}

In [3]:
# Initialising dominance matrices
dominance_matrix_west_1 = pd.DataFrame([0][0], conferences["west"], columns=conferences["west"])
dominance_matrix_east_1 = pd.DataFrame([0][0], conferences["east"], columns=conferences["east"])

In [4]:
wins_east = {}

# Looping through all games
for idx, line in data.iterrows():
    
    # Getting team names from data
    team1 = line['Visitor/Neutral']
    team2 = line['Home/Neutral']
    
    # Checking if both teams are in the Eastern conference, if not, skip this game
    if team1 not in conferences["east"]: continue
    if team2 not in conferences["east"]: continue
    
    # Getting scores from data
    score1 = line['PTS']
    score2 = line['PTS.1']
    
    # Initialising key-value data structure with team1 and team2 
    # to store point margins if not already there
    if team1 not in wins_east: wins_east[team1] = {}
    if team2 not in wins_east: wins_east[team2] = {}
    if team2 not in wins_east[team1]: wins_east[team1][team2] = 0
    if team1 not in wins_east[team2]: wins_east[team2][team1] = 0
    
    # Adding point margin to winning team's score in data structure
    if score1 > score2: wins_east[team1][team2] += score1 - score2
    if score2 > score1: wins_east[team2][team1] += score2 - score1
    
    ### Old matrix method (Model 1a) 
    # if score1 > score2: wins_east[team1][team2] += 1
    # if score2 > score1: wins_east[team2][team1] += 1



wins_west = {}

# Looping through all games
for idx, line in data.iterrows():
    
    # Getting team names from data
    team1 = line['Visitor/Neutral']
    team2 = line['Home/Neutral']
    
    # Checking if both teams are in the Western conference, if not, skip this game
    if team1 not in conferences["west"]: continue
    if team2 not in conferences["west"]: continue
    
    # Getting scores from data
    score1 = line['PTS']
    score2 = line['PTS.1']
    
    # Initialising key-value data structure with team1 and team2
    # to store point margins if not already there
    if team1 not in wins_west: wins_west[team1] = {}
    if team2 not in wins_west: wins_west[team2] = {}
    if team2 not in wins_west[team1]: wins_west[team1][team2] = 0
    if team1 not in wins_west[team2]: wins_west[team2][team1] = 0
    
    # Adding point margin to winning team's score in data structure
    if score1 > score2: wins_west[team1][team2] += score1 - score2
    if score2 > score1: wins_west[team2][team1] += score2 - score1
    
    ### Old matrix method (Model 1a) 
    # if score1 > score2: wins_east[team1][team2] += 1
    # if score2 > score1: wins_east[team2][team1] += 1

In [5]:
# Setting weighting parameters for dominance vectors
v2_weight=0.1
v3_weight=0.01

In [ ]:
# Looping through earlier data structure for Eastern conference
for team1 in wins_east:
    for team2 in wins_east[team1]:

        # If team1 has a bigger point margin overall than team2
        if wins_east[team1][team2] > wins_east[team2][team1]:

            # Populating dominance matrix
            dominance_matrix_east_1.at[team1, team2] = 1
            dominance_matrix_east_1.at[team2, team1] = 0
        
        # If team1 has a smaller point margin overall than team2
        elif wins_east[team1][team2] < wins_east[team2][team1]:
            
            # Populating dominance matrix
            dominance_matrix_east_1.at[team1, team2] = 0
            dominance_matrix_east_1.at[team2, team1] = 1
        
        # If team1 has the same point margin overall as team2
        elif wins_east[team1][team2] == wins_east[team2][team1]:

            # Populating dominance matrix
            dominance_matrix_east_1.at[team1, team2] = 0.5
            dominance_matrix_east_1.at[team2, team1] = 0.5
            
# Outputting dominance matrix (east)
dominance_matrix_east_1


In [ ]:
# Squaring and Cubing dominance matrix for second and third order dominance matrices with .dot() function
dominance_matrix_east_2 = dominance_matrix_east_1.dot(dominance_matrix_east_1)
dominance_matrix_east_3 = dominance_matrix_east_2.dot(dominance_matrix_east_1)

# Initialing matrix for dominace vectors
dominance_vector_matrix_east = pd.DataFrame()

# Adding rows of dominance matrices and populating dominace vector matrix
dominance_vector_matrix_east["v1"]=dominance_matrix_east_1.sum(axis=1)
dominance_vector_matrix_east["v2"]=dominance_matrix_east_2.sum(axis=1)
dominance_vector_matrix_east["v3"]=dominance_matrix_east_3.sum(axis=1)

# Calculating overall ranking score from weighting parameters and adding column to dominace vector matrix
dominance_vector_matrix_east["score"] = dominance_matrix_east_1.sum(axis=1) + v2_weight*dominance_matrix_east_2.sum(axis=1) + v3_weight*dominance_matrix_east_3.sum(axis=1)

# Sorting teams by score to determine ranking
dominance_vector_matrix_east = dominance_vector_matrix_east.sort_values("score", ascending=False)

# Outputting ranking (east)
dominance_vector_matrix_east

In [ ]:
# Looping through earlier data structure for Western conference
for team1 in wins_west:
    for team2 in wins_west[team1]:

        # If team1 has a bigger point margin overall than team2
        if wins_west[team1][team2] > wins_west[team2][team1]:

            # Populating dominance matrix
            dominance_matrix_west_1.at[team1, team2] = 1
            dominance_matrix_west_1.at[team2, team1] = 0
        
        # If team1 has a smaller point margin overall than team2
        elif wins_west[team1][team2] < wins_west[team2][team1]:
            
            # Populating dominance matrix
            dominance_matrix_west_1.at[team1, team2] = 0
            dominance_matrix_west_1.at[team2, team1] = 1
        
        # If team1 has the same point margin overall as team2
        elif wins_west[team1][team2] == wins_west[team2][team1]:

            # Populating dominance matrix
            dominance_matrix_west_1.at[team1, team2] = 0.5
            dominance_matrix_west_1.at[team2, team1] = 0.5

# Outputting dominance matrix (west)
dominance_matrix_west_1

In [ ]:
# Squaring and Cubing dominance matrix for second and third order dominance matrices with .dot() function
dominance_matrix_west_2 = dominance_matrix_west_1.dot(dominance_matrix_west_1)
dominance_matrix_west_3 = dominance_matrix_west_2.dot(dominance_matrix_west_1)

# Initialing matrix for dominace vectors
dominance_vector_matrix_west = pd.DataFrame()

# Adding rows of dominance matrices and populating dominace vector matrix
dominance_vector_matrix_west["v1"] = dominance_matrix_west_1.sum(axis=1)
dominance_vector_matrix_west["v2"] = dominance_matrix_west_2.sum(axis=1)
dominance_vector_matrix_west["v3"] = dominance_matrix_west_3.sum(axis=1)

# Calculating overall ranking score from weighting parameters and adding column to dominace vector matrix
dominance_vector_matrix_west["score"] = dominance_matrix_west_1.sum(axis=1) + v2_weight*dominance_matrix_west_2.sum(axis=1) + v3_weight*dominance_matrix_west_3.sum(axis=1)

# Sorting teams by score to determine ranking
dominance_vector_matrix_west = dominance_vector_matrix_west.sort_values("score", ascending=False)

# Outputting ranking (west)
dominance_vector_matrix_west

In [ ]:
##### STOP HERE AS CORRECT TEAM IS NOT INCLUDED #####


# Finding players in the teams with a seed greater than or equal to 3
mvp_candidates = players[(players["TEAM"].isin(list(dominance_vector_matrix_west.head(3).index)) | players["TEAM"].isin(list(dominance_vector_matrix_east.head(3).index)))]

#Initialising fantasy points array
fantasy_points = []

# Calculating fantasy points for each eligible player
for idx, player in mvp_candidates.iterrows():
    fantasy_points.append(3*player["3PM"]+2*(player["FGM"]-player["3PM"])+player["FTM"]+1.2*player["REB"]+1.5*player["AST"]+2*player["STL"]+2*player["BLK"]-player["TO"])

# Adding fantasy points row to MVP candidates matrix
mvp_candidates["Fantasy Points"] = fantasy_points

# Ordering MVP candidates by Fantasy points to determine recipient
mvp_candidates = mvp_candidates.sort_values(by="Fantasy Points", ascending=False)

# Outputting MVP candidates matrix
mvp_candidates

In [11]:
# Outputting MVP
print("Season MVP from seed <= 3 is " + list(mvp_candidates.head(1)["PLAYER"])[0])

Season MVP from seed <= 3 is Kevin Durant
